In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [ ]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [ ]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import *
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import *
from desc.vmec_utils import *
from desc.input_reader import *
from desc.continuation import *

print_backend_info()

In [ ]:
eq = get("precise_QA")
eq.R_lmn.shape

In [ ]:
from desc.optimize.utils import compute_jac_scale

method = "direct1"
tconds = []
jconds = []
jscaleconds = []
ress = np.arange(8, 16)
for res in ress:
    eq.change_resolution(L=res, M=res, N=res, L_grid=2*res, M_grid=2*res, N_grid=2*res)
    grid = ConcentricGrid(L=eq.L_grid, M=eq.M_grid, N=eq.N_grid, NFP=eq.NFP, sym=eq.sym)
    transform = Transform(grid=grid, basis=eq.R_basis, method=method)
    if method in ["direct2"]:
        A = transform.matrices[method][0]
    elif method in ["fft"]:
        A = transform.matrices[method][0][0]
    else:
        A = transform.matrices[method][0][0][0]
    tcond = np.linalg.cond(A)
    tconds.append(tcond)
    print(f"Condition number for F-Z transform LMN={res:^4} ===>  {tcond:.2e}")
    obj = ObjectiveFunction(ForceBalance(eq), jac_chunk_size=10)
    cons = get_fixed_boundary_constraints(eq)
    cons = maybe_add_self_consistency(eq, cons)
    con = ObjectiveFunction(cons)
    obj = LinearConstraintProjection(obj, con)
    obj.build(verbose=0)
    J = obj.jac_scaled_error(obj.x())
    scale, scale_inv = compute_jac_scale(J)
    Jcond = np.linalg.cond(J)
    Jscalecond = np.linalg.cond(J*scale)
    jconds.append(Jcond)
    jscaleconds.append(Jscalecond)
    print(f"Condition number for Jacobian LMN={res:^4} ===>  {Jcond:.2e}")
    print(f"Condition number for preconditioned Jacobian LMN={res:^4} ===>  {Jscalecond:.2e}")

In [ ]:
A.shape

In [ ]:
grid = LinearGrid(L=1, M=4, N=4)
obj = ObjectiveFunction(ForceBalance(eq))
cons = get_fixed_boundary_constraints(eq)
cons = maybe_add_self_consistency(eq, cons)
con = ObjectiveFunction(cons)
obj = LinearConstraintProjection(obj, con)
obj.build()

In [ ]:
J = obj.jac_scaled_error(obj.x())

In [ ]:
cond = np.linalg.cond(J)
print(f"{cond:.2e}")